<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Todos" data-toc-modified-id="Todos-1">Todos</a></span></li><li><span><a href="#1.-Setup" data-toc-modified-id="1.-Setup-2">1. Setup</a></span></li><li><span><a href="#2.-Get-data" data-toc-modified-id="2.-Get-data-3">2. Get data</a></span><ul class="toc-item"><li><span><a href="#2.a.-Create-feature-matrix" data-toc-modified-id="2.a.-Create-feature-matrix-3.1">2.a. Create feature matrix</a></span></li><li><span><a href="#2.b.-Create-label-vector" data-toc-modified-id="2.b.-Create-label-vector-3.2">2.b. Create label vector</a></span></li><li><span><a href="#2.c.-Create-report" data-toc-modified-id="2.c.-Create-report-3.3">2.c. Create report</a></span></li><li><span><a href="#2.d.-Split-data-&amp;-shuffle" data-toc-modified-id="2.d.-Split-data-&amp;-shuffle-3.4">2.d. Split data &amp; shuffle</a></span></li></ul></li><li><span><a href="#2.-Pipeline" data-toc-modified-id="2.-Pipeline-4">2. Pipeline</a></span><ul class="toc-item"><li><span><a href="#2.a.-Metrics-wrap-up" data-toc-modified-id="2.a.-Metrics-wrap-up-4.1">2.a. Metrics wrap up</a></span><ul class="toc-item"><li><span><a href="#2.a.1.-Confusion-matrix-metrics" data-toc-modified-id="2.a.1.-Confusion-matrix-metrics-4.1.1">2.a.1. Confusion matrix metrics</a></span><ul class="toc-item"><li><span><a href="#True-negative-(TN)" data-toc-modified-id="True-negative-(TN)-4.1.1.1">True negative (TN)</a></span></li><li><span><a href="#False-negative-(FN)" data-toc-modified-id="False-negative-(FN)-4.1.1.2">False negative (FN)</a></span></li><li><span><a href="#True-positive-(TP)" data-toc-modified-id="True-positive-(TP)-4.1.1.3">True positive (TP)</a></span></li><li><span><a href="#False-positive-(FP)" data-toc-modified-id="False-positive-(FP)-4.1.1.4">False positive (FP)</a></span></li></ul></li><li><span><a href="#2.a.2.-Metrics" data-toc-modified-id="2.a.2.-Metrics-4.1.2">2.a.2. Metrics</a></span><ul class="toc-item"><li><span><a href="#Precision" data-toc-modified-id="Precision-4.1.2.1">Precision</a></span></li><li><span><a href="#Recall-or-True-Positive-Rate-(TPR)" data-toc-modified-id="Recall-or-True-Positive-Rate-(TPR)-4.1.2.2">Recall or True Positive Rate (TPR)</a></span></li><li><span><a href="#False-Positive-Rate-(FPR)" data-toc-modified-id="False-Positive-Rate-(FPR)-4.1.2.3">False Positive Rate (FPR)</a></span></li><li><span><a href="#Receiver-Operating-Characteristic-curve-(ROC)" data-toc-modified-id="Receiver-Operating-Characteristic-curve-(ROC)-4.1.2.4">Receiver Operating Characteristic curve (ROC)</a></span></li><li><span><a href="#In-our-case" data-toc-modified-id="In-our-case-4.1.2.5">In our case</a></span></li></ul></li></ul></li><li><span><a href="#2.b.-Pipeline-creation" data-toc-modified-id="2.b.-Pipeline-creation-4.2">2.b. Pipeline creation</a></span></li></ul></li><li><span><a href="#4.-Optimization" data-toc-modified-id="4.-Optimization-5">4. Optimization</a></span><ul class="toc-item"><li><span><a href="#4.a.-Gradient-boosting" data-toc-modified-id="4.a.-Gradient-boosting-5.1">4.a. Gradient boosting</a></span></li><li><span><a href="#4.b.-XGBoost" data-toc-modified-id="4.b.-XGBoost-5.2">4.b. XGBoost</a></span></li></ul></li><li><span><a href="#4.c.-Ensemble-learning" data-toc-modified-id="4.c.-Ensemble-learning-6">4.c. Ensemble learning</a></span></li><li><span><a href="#5.-Diagnostic" data-toc-modified-id="5.-Diagnostic-7">5. Diagnostic</a></span><ul class="toc-item"><li><span><a href="#5.a.-Feature-importances" data-toc-modified-id="5.a.-Feature-importances-7.1">5.a. Feature importances</a></span></li><li><span><a href="#5.b.-Learning-curves" data-toc-modified-id="5.b.-Learning-curves-7.2">5.b. Learning curves</a></span></li><li><span><a href="#5.c.-ROC-curves" data-toc-modified-id="5.c.-ROC-curves-7.3">5.c. ROC curves</a></span></li><li><span><a href="#5.d.-Precision-recall-vs-threshold" data-toc-modified-id="5.d.-Precision-recall-vs-threshold-7.4">5.d. Precision-recall vs threshold</a></span></li><li><span><a href="#5.e.-Confusion-matrix" data-toc-modified-id="5.e.-Confusion-matrix-7.5">5.e. Confusion matrix</a></span></li></ul></li><li><span><a href="#6.-Evaluation" data-toc-modified-id="6.-Evaluation-8">6. Evaluation</a></span><ul class="toc-item"><li><span><a href="#6.a.-Confusion-matrix" data-toc-modified-id="6.a.-Confusion-matrix-8.1">6.a. Confusion matrix</a></span></li><li><span><a href="#6.b.-Classification-scalar-metrics" data-toc-modified-id="6.b.-Classification-scalar-metrics-8.2">6.b. Classification scalar metrics</a></span></li></ul></li><li><span><a href="#7.-Summary" data-toc-modified-id="7.-Summary-9">7. Summary</a></span></li></ul></div>

# Project Fatigue
- GitHub [link](https://github.com/romainmartinez/fatigue)
- Plotly [link](https://plot.ly/organize/romainmartinez:97)

## Todos
- ensemble methods

## 1. Setup

In [1]:
# Common imports
import scipy.io as sio
import numpy as np
import warnings
import os

# Figures
OFFLINE = True
if OFFLINE:
    import plotly.offline as py
    py.init_notebook_mode(connected=True)
else:
    import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
base_layout = go.Layout(hovermode='closest', font=dict(size=14))

# to make this notebook's output stable across runs
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 2. Get data

In [2]:
DATA_PATH = os.path.join('.', 'data/')
DATA_FILE = 'DatabaseRPT.mat'
mat = sio.loadmat(os.path.join(DATA_PATH, DATA_FILE))['DataBaseRPT'][0, 0]

- label
    - `Y` (1, 162): 1 = prefatigue, 2 = fatigue

- used features
    - `AllX` (24, 162): 24 (6 DoF x 4 variables) x 162 (81 participants x 2 time points).
    - `CAssignAll` (1, 24): AllX column assignment

    - `Sex` (1, 162)
    - `Endurance` (1, 162)

- not used
    - `BestX` (7, 162): 7 (variables with SRM>0.8) x 162 (81 participants x 2 time points) matrix. Contains data only for the most responsive variables (SRM>0.8).
    - `CAssignBest` (1, 7): BestX column assignment.
    - `Age` (1, 162): too much NaN.
    - `Height` (1, 162): too much NaN.
    - `Weight` (1, 162) Too much NaN.
    - `SubjectKey` (1, 162): useless.
    - `SID` (1, 162): useless.

### 2.a. Create feature matrix

In [3]:
col_names = [i[0] for i in mat['CAssignAll'].flatten()]
# find variable with SRM > .8
srm_names = np.array([i[0] for i in mat['CAssignBest'].flatten()])
srm_idx = np.in1d(col_names, srm_names)

# add `AllX`
X = mat['AllX'].T

SEX = False
if SEX:
    # add `Sex`
    X = np.c_[X, mat['Sex'].T]
    col_names.append('Sex')

SUM_ZSCORE = True
if SUM_ZSCORE:
    # add sum of zscore
    from sklearn.preprocessing import StandardScaler
    sum_zscore_all = StandardScaler().fit_transform(X).sum(axis=1)
    sum_zscore_srm = StandardScaler().fit_transform(X)[:, srm_idx].sum(axis=1)
    X = np.c_[X, sum_zscore_all, sum_zscore_srm]
    col_names.extend(['sum_zscore_all', 'sum_zscore_srm'])
    srm_idx = np.append(srm_idx, [False, False])

col_names = np.array(col_names)

### 2.b. Create label vector

In [4]:
from sklearn.preprocessing import LabelBinarizer
y = mat['Y'].T
y = LabelBinarizer().fit_transform(y)

### 2.c. Create report

In [5]:
import os.path
import pandas as pd

# create dataframe
df = pd.DataFrame(X, columns=col_names)
df['fatigue (label)'] = y

# generate pandas report
REPORT_FILENAME = './pandas_report.html'
if not os.path.isfile(REPORT_FILENAME):
    import pandas_profiling
    report = pandas_profiling.ProfileReport(df)
    report.to_file('./pandas_report.html')

### 2.d. Split data & shuffle

In [6]:
# split data & shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED)

In [7]:
df.head()

,AveElbFlexMean,ROMElbFlexMean,AveElbFlexSD,ROMElbFlexSD,AveShPlaneMean,ROMShPlaneMean,AveShPlaneSD,ROMShPlaneSD,AveShEleMean,ROMShEleMean,...,ROMTrYMean,AveTrYSD,ROMTrYSD,AveTrZMean,ROMTrZMean,AveTrZSD,ROMTrZSD,sum_zscore_all,sum_zscore_srm,fatigue (label)
0,105.983144,63.339618,2.219401,1.794134,23.518597,13.624799,1.338393,2.357813,80.274017,2.942674,...,13.629558,1.112671,1.977798,-0.550047,2.671822,0.228494,0.711787,-10.074628,-2.583470,0
1,106.166216,61.875226,2.239310,2.548458,22.851141,11.726534,1.182490,1.401127,77.897777,4.062635,...,15.078508,0.910697,1.003788,-0.208865,1.242553,0.541000,0.496108,-10.663077,-1.027980,1
2,100.914284,60.930029,1.482458,1.167593,27.289316,12.291697,1.223189,2.452705,83.932823,2.886935,...,14.311074,0.979720,2.070705,0.096174,0.934767,0.488540,0.317386,-12.833948,-1.552097,0
3,105.066549,59.524098,2.364182,1.849991,23.083816,7.519415,1.510000,3.503819,78.701025,2.997990,...,16.178023,1.418294,3.521951,-0.544067,1.885480,0.610577,0.851287,-5.036257,-1.688547,1
4,93.259333,81.531138,2.610755,3.632922,42.387152,27.789377,1.605530,2.027628,71.280986,8.000008,...,8.815273,0.709582,1.250228,-0.123442,1.065910,0.415764,0.404298,-1.780486,-3.241088,0


## 2. Pipeline

### 2.a. Metrics wrap up

#### 2.a.1. Confusion matrix metrics

| Predicted | Real: 1 | Real: 0 |
|-----------|---------|---------|
|    1      |  TP     |    FP   |
|    0      |  FN     |    TN   |

##### True negative (TN)
We *correctly* predict that the class is *negative*.

##### False negative (FN)
We *incorrectly* predict that the class is *negative*.

##### True positive (TP)
We *correctly* predict that the class is *positive*.

##### False positive (FP)
We *incorrectly* predict that the class is *positive*.

#### 2.a.2. Metrics

##### Precision
$$\frac{TP}{\text{n}^o \text{predicted positives}} = \frac{TP}{TP + FP}$$

- high precision = less _false positive_
- _example_: getting $0.8$ precision means that when it claims to detect a label, it is right $80$% of the time

##### Recall or True Positive Rate (TPR)
$$\frac{TP}{\text{n}^o \text{actual positives}} = \frac{TP}{TP + FN}$$

- high recall = less _false negative_
- _example_: getting $0.8$ recall means that it detect $80$% of the label

##### False Positive Rate (FPR)
$$\frac{FP}{\text{n}^o \text{actual negatives}} = \frac{FP}{FP + TN}$$

- high FPR = more _negative data points missclassified_
- proportion of negative data points that are mistakenly considered as positive, with respect to all negative data points

##### Receiver Operating Characteristic curve (ROC)
Compute _TPR_ and _FPR_ for many different threshold and plot FPR vs. TPR

##### In our case
We want to optimize our algorithm for the area Under the ROC curve ([wiki](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve), [stackexchange](https://stats.stackexchange.com/a/132832)).

### 2.b. Pipeline creation

In [8]:
def get_categorical_cols(X):
    return X[:, col_names == 'Sex']


def get_numerical_cols(X):
    return X[:, col_names != 'Sex']


def get_high_srm_cols(X, srm_idx=srm_idx):
    return X[:, srm_idx]

In [9]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

# categorical
from sklearn.preprocessing import OneHotEncoder

# numerical
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures

pipeline_categorical = Pipeline([
    ('selector', FunctionTransformer(get_categorical_cols, validate=False)),
    ('encoder', OneHotEncoder(sparse=False))
])

pipeline_numerical = Pipeline([
    ('selector', FunctionTransformer(get_numerical_cols, validate=False)),
    ('selecthighsrm', FunctionTransformer(get_high_srm_cols, validate=False)),
    ('scale', StandardScaler()),
    ('polyfeatures', PolynomialFeatures(degree=2))
])

if SEX:
    pipeline_preprocessing = FeatureUnion([
        ('categorical', pipeline_categorical),
        ('numerical', pipeline_numerical)
    ])
else:
    pipeline_preprocessing = FeatureUnion([
        ('numerical', pipeline_numerical)
    ])

In [10]:
from sklearn.model_selection import GridSearchCV
import pprint


class GridSearchWrapper:
    def __init__(self, base_pipeline, model, param_grid, scoring):
        self.base_pipeline = base_pipeline
        self.model = model
        self.param_grid = param_grid
        self.scoring = scoring
    
    def fit(self, X, y):
        pipeline = Pipeline([
            ('preprocessing', self.base_pipeline),
            ('pca', PCA(n_components=0.99, svd_solver='full')),
            ('classifier', self.model)
        ])
        
        self.grid_search = GridSearchCV(pipeline,
                                        self.param_grid,
                                        scoring=self.scoring,
                                        cv=5,
                                        n_jobs=-1,
                                        verbose=1)
        self.grid_search.fit(X, y)
    
    def print_scores(self):
        print(f'best score ({self.scoring}): {self.grid_search.best_score_:.3f}')
        print('\nbest params:')
        pprint.pprint(self.grid_search.best_params_)

In [11]:
# wide
# param_grid = {
#     'preprocessing__numerical__selecthighsrm': [None, FunctionTransformer(get_high_srm_cols, validate=False)],
#     'preprocessing__numerical__polyfeatures': [None, PolynomialFeatures(degree=1), PolynomialFeatures(degree=2),
#                                                PolynomialFeatures(degree=3)],
#     'preprocessing__numerical__scale': [None, StandardScaler()],
#     'pca': [None, PCA(n_components=0.99, svd_solver='full')]
# }

# selected
param_grid = {
    'preprocessing__numerical__selecthighsrm': [None, FunctionTransformer(get_high_srm_cols, validate=False)],
    'preprocessing__numerical__polyfeatures': [None],
    'preprocessing__numerical__scale': [None, StandardScaler()],
    'pca': [None]
}

scoring = 'roc_auc'
model = {}

## 4. Optimization

### 4.a. Gradient boosting

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

grad_param = {
    'learning_rate': 0.001,
    'max_depth': 1,
    'max_features': 0.75,
    'min_samples_leaf': 7,
    'min_samples_split': 16,
    'n_estimators': 100,
    'subsample': 0.3,
    'random_state': RANDOM_SEED
}

wrapper = GridSearchWrapper(
    base_pipeline=pipeline_preprocessing,
    model=GradientBoostingClassifier(**grad_param),
    param_grid=param_grid,
    scoring=scoring)

wrapper.fit(X_train, y_train.ravel())

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done   5 out of  20 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.3s finished


In [13]:
wrapper.print_scores()
model['gradboost'] = wrapper.grid_search.best_estimator_

best score (roc_auc): 0.916

best params:
{'pca': None,
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': None}


### 4.b. XGBoost

https://jessesw.com/XG-Boost/

In [14]:
from xgboost import XGBClassifier

xgboost_param = {
    'learning_rate': 0.06,
    'max_depth': 4,
    'max_features': 0.75,
    'min_samples_leaf': 7,
    'min_samples_split': 16,
    'min_child_weight': 1,
    'colsample_bytree': 0.7,
    'n_estimators': 120,
    'subsample': 0.9,
    'reg_lambda': 0.7,
    'reg_alpha': 0.5,
    'gamma': 0.2,
    'random_state': RANDOM_SEED,
    'silent': False
}

warnings.simplefilter('ignore')
wrapper = GridSearchWrapper(
    base_pipeline=pipeline_preprocessing,
    model=XGBClassifier(**xgboost_param),
    param_grid=param_grid,
    scoring=scoring)

wrapper.fit(X_train, y_train.ravel())
warnings.simplefilter('default')

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done   5 out of  20 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


In [15]:
wrapper.print_scores()
model['xgboost'] = wrapper.grid_search.best_estimator_

best score (roc_auc): 0.891

best params:
{'pca': None,
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': None}


## 4.c. Ensemble learning

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier

clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=RANDOM_SEED)),
        ('naive', GaussianNB()),
        ('forest', RandomForestClassifier(random_state=RANDOM_SEED)),
        ('gradient',  GradientBoostingClassifier(**grad_param)),
        ('xgboost',  XGBClassifier(**xgboost_param))
    ], voting='soft', weights=[1, 1, 2, 3, 3])

warnings.simplefilter('ignore')
wrapper = GridSearchWrapper(
    base_pipeline=pipeline_preprocessing,
    model=clf,
    param_grid=param_grid,
    scoring=scoring)

wrapper.fit(X_train, y_train.ravel())
warnings.simplefilter('default')

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done   5 out of  20 | elapsed:    0.2s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.4s finished


In [17]:
wrapper.print_scores()
model['xgboost'] = wrapper.grid_search.best_estimator_

best score (roc_auc): 0.878

best params:
{'pca': None,
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': None}


## 5. Diagnostic

In [18]:
from sklearn.model_selection import cross_val_predict

# final_model = model['gradboost']
final_model = model['xgboost']

warnings.simplefilter('ignore')
cv_predict_proba = cross_val_predict(
    final_model, X_train, y_train.ravel(), cv=5, method='predict_proba')

cv_predict = cross_val_predict(
    final_model, X_train, y_train.ravel(), cv=5, method='predict')
warnings.simplefilter('default')

### 5.a. Feature importances

In [19]:
def plot_feature_importances(importances, col_names):
    # sort feature
    indices = np.argsort(importances)

    trace = go.Bar(
        x=importances[indices],
        y=col_names[indices],
        marker=dict(color='red'),
        orientation='h',
        opacity=0.5)

    layout = base_layout.copy()
    layout.update(
        dict(
            title='Feature importances',
            xaxis=dict(
                title='Feature importances (%)', showline=True, linewidth=1.5),
            yaxis=dict(showline=True, linewidth=1.5)))

    # adjust y axis
    layout['yaxis'].update(nticks=importances.shape[0])
    layout.update(margin=go.Margin(t=80, b=80, l=150, r=80, pad=0))

    fig = dict(data=[trace], layout=layout)
    return fig

In [20]:
# update col_names given one hot encoding
if SEX:
    feature_names = col_names.copy()
    feature_names[-1] = 'Sex_0'
    feature_names = np.append(col_names, 'Sex_1')

importances = final_model.named_steps['classifier'].feature_importances_ * 100

f = plot_feature_importances(importances, col_names)
py.iplot(f, filename='fatigue/feature_importances')

AttributeError: 'VotingClassifier' object has no attribute 'feature_importances_'

### 5.b. Learning curves

In [21]:
from sklearn.model_selection import learning_curve


def plot_learning_curve(
        estimator,
        X,
        y,
        scoring='roc_auc',
        cv=None,
        n_jobs=1,
        train_sizes=np.linspace(.1, 1.0, 5),
):
    warnings.simplefilter('ignore')  # ignore scikit-learn warnings
    train_sizes, train_scores, test_scores = learning_curve(
        estimator,
        X,
        y,
        scoring=scoring,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes)
    warnings.simplefilter('default')

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)

    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    trace = []

    # training mean
    trace.append(
        go.Scatter(
            x=train_sizes,
            y=train_scores_mean,
            marker=dict(color='red'),
            name='Training score'))

    # training std
    trace.append(
        go.Scatter(
            x=train_sizes,
            y=train_scores_mean + train_scores_std,
            mode='lines',
            line=dict(color='red', width=1),
            showlegend=False))

    trace.append(
        go.Scatter(
            x=train_sizes,
            y=train_scores_mean - train_scores_std,
            mode='lines',
            line=dict(color='red', width=1),
            fill='tonexty',
            showlegend=False))

    # test mean
    trace.append(
        go.Scatter(
            x=train_sizes,
            y=test_scores_mean,
            marker=dict(color='green'),
            name='Cross-validation score'))

    # test std
    trace.append(
        go.Scatter(
            x=train_sizes,
            y=test_scores_mean + test_scores_std,
            mode='lines',
            line=dict(color='green', width=1),
            showlegend=False))

    trace.append(
        go.Scatter(
            x=train_sizes,
            y=test_scores_mean - test_scores_std,
            mode='lines',
            line=dict(color='green', width=1),
            fill='tonexty',
            showlegend=False))

    data = [itrace for itrace in trace]
    layout = base_layout.copy()
    layout.update(
        dict(
            title='Learning curves',
            xaxis=dict(
                title='Training examples', showline=True, linewidth=1.5),
            yaxis=dict(title='Score (ROC AUC)', showline=True, linewidth=1.5)))
    fig = dict(data=data, layout=layout)
    return fig

In [22]:
l = plot_learning_curve(
    estimator=final_model,
    X=X_train,
    y=y_train.ravel(),
    scoring=scoring,
    cv=5,
    train_sizes=np.linspace(.1, 1.0, 10))
py.iplot(l, filename='fatigue/learning_curve')

### 5.c. ROC curves

In [23]:
from sklearn.metrics import roc_curve, roc_auc_score


def plot_roc_curve(y_true, y_pred):
    fpr, tpr, auc_thresholds = roc_curve(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)

    trace_roc = go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        line=dict(color='gray'),
        name=f'ROC curve (AUC = {roc_auc:.2f})',
        fill='tozeroy')

    trace_random = go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        line=dict(color='black', dash='dash'),
        name='Random')

    layout = base_layout.copy()
    layout.update(
        dict(
            title=
            f'Receiver operating characteristic curve (roc auc = {roc_auc:.2f})',
            xaxis=dict(
                title='False positive rate', showline=True, linewidth=1.5),
            yaxis=dict(
                title='True positive rate',
                showline=True,
                linewidth=1.5,
                zeroline=False)))
    fig = dict(data=[trace_roc, trace_random], layout=layout)
    return fig

In [24]:
r = plot_roc_curve(y_train, cv_predict_proba[:, 1])
py.iplot(r, filename='fatigue/roc_curve')

### 5.d. Precision-recall vs threshold

In [25]:
from sklearn.metrics import precision_recall_curve


def precision_recall_vs_threshold(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)

    trace_precision = go.Scatter(
        x=thresholds,
        y=precision[:-1],
        mode='lines',
        line=dict(color='blue'),
        name='Precision')

    trace_recall = go.Scatter(
        x=thresholds,
        y=recall[:-1],
        mode='lines',
        line=dict(color='green'),
        name='Recall')

    layout = base_layout.copy()
    layout.update(
        dict(
            title=
            'Precision and recall scores as a function of the decision threshold',
            xaxis=dict(
                title='Decision threshold', showline=True, linewidth=1.5),
            yaxis=dict(title='Score', showline=True, linewidth=1.5)))
    fig = dict(data=[trace_precision, trace_recall], layout=layout)
    return fig

In [26]:
t = precision_recall_vs_threshold(
    y_true=y_train, y_pred=cv_predict_proba[:, 1])
py.iplot(t, filename='fatigue/precision_recall_vs_threshold')

### 5.e. Confusion matrix

In [27]:
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(y_true, y_pred, in_percentage=True):
    cm = confusion_matrix(y_true, y_pred)
    if in_percentage:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    colorscale = [[0, '#2d3436'], [1, '#dfe6e9']]

    fig = ff.create_annotated_heatmap(
        z=cm.astype(int),
        x=['True', 'False'],
        y=['True', 'False'],
        showscale=True,
        zmin=0,
        zmax=100,
        colorscale=colorscale,
        colorbar=dict(title='Percentage', titleside='right'))

    fig['layout'].update(base_layout)
    fig['layout'].update(
        dict(
            title='confusion matrix',
            xaxis=dict(title='Predicted values', side='bottom'),
            yaxis=dict(title='True values', autorange='reversed')))

    return fig

In [28]:
m = plot_confusion_matrix(y_true=y_train, y_pred=cv_predict)
py.iplot(m, filename='fatigue/confusion_matrix_train')

## 6. Evaluation

In [29]:
warnings.simplefilter('ignore')
y_pred = final_model.predict(X_test)
warnings.simplefilter('default')

### 6.a. Confusion matrix

In [30]:
m = plot_confusion_matrix(y_true=y_test, y_pred=y_pred)
py.iplot(m, filename='fatigue/confusion_matrix_test')

### 6.b. Classification scalar metrics

In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score


def table_classification_report(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    data_matrix = [['Metric', 'Value'],
                   ['Precision', f'{precision:.2f}'],
                   ['Recall', f'{recall:.2f}'],
                   ['F1 score', f'{f1:.2f}'],
                   ['ROC AUC', f'{roc_auc:.2f}']]

    table = ff.create_table(data_matrix)
    table['layout'].update(font=dict(size=14))
    return table

In [32]:
table = table_classification_report(y_true=y_test, y_pred=y_pred)

py.iplot(table, filename='fatigue/clf_report')

## 7. Summary